# Day 3: No matter how you slice it

## Part 1 
 
[source](https://adventofcode.com/2018/day/3)

The Elves managed to locate the chimney-squeeze prototype fabric for Santa's suit (thanks to someone who helpfully wrote its box IDs on the wall of the warehouse in the middle of the night). Unfortunately, anomalies are still affecting them - nobody can even agree on how to cut the fabric.

The whole piece of fabric they're working on is a very large square - at least 1000 inches on each side.

Each Elf has made a claim about which area of fabric would be ideal for Santa's suit. All claims have an ID and consist of a single rectangle with edges parallel to the edges of the fabric. Each claim's rectangle is defined as follows:

    The number of inches between the left edge of the fabric and the left edge of the rectangle.
    The number of inches between the top edge of the fabric and the top edge of the rectangle.
    The width of the rectangle in inches.
    The height of the rectangle in inches.

A claim like #123 @ 3,2: 5x4 means that claim ID 123 specifies a rectangle 3 inches from the left edge, 2 inches from the top edge, 5 inches wide, and 4 inches tall. Visually, it claims the square inches of fabric represented by # (and ignores the square inches of fabric represented by .) in the diagram below:

```
...........
...........
...#####...
...#####...
...#####...
...#####...
...........
...........
...........
```

The problem is that many of the claims overlap, causing two or more claims to cover part of the same areas. For example, consider the following claims:

```
#1 @ 1,3: 4x4
#2 @ 3,1: 4x4
#3 @ 5,5: 2x2
````

Visually, these claim the following areas:

```
........
...2222.
...2222.
.11XX22.
.11XX22.
.111133.
.111133.
........
```

The four square inches marked with X are claimed by both 1 and 2. (Claim 3, while adjacent to the others, does not overlap either of them.)

If the Elves all proceed with their own plans, none of them will have enough fabric. **How many square inches of fabric are within two or more claims**?

---

In [1]:
import re

import pandas as pd
import numpy as np

### Fetching the data

In [2]:
# get the puzzle input
! cd input && wget https://adventofcode.com/2018/day/3/input

--2018-12-03 21:19:27--  https://adventofcode.com/2018/day/3/input
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving adventofcode.com (adventofcode.com)... 34.226.237.180, 52.55.170.144
Connecting to adventofcode.com (adventofcode.com)|34.226.237.180|:443... connected.
HTTP request sent, awaiting response... 400 Bad Request
2018-12-03 21:19:28 ERROR 400: Bad Request.



In [3]:
# try again getting the input
!curl https://adventofcode.com/2018/day/3/input

Puzzle inputs differ by user.  Please log in to get your puzzle input.


In [4]:
# AHA, lets just copy it from the site :D 
with open ('input/day3.txt', 'r') as f:
    data = f.readlines()
data[:2]

['#1 @ 37,526: 17x23\n', '#2 @ 75,649: 23x11\n']

>Create a regex string to retrieve the useful data

In [5]:
# https://regex101.com/
regex = r"(\d+) ..(\d+),(\d+): (\d+)x(\d+)"

match = re.search(regex,data[0])
for i in range(1,6):
    print(match.group(i))

1
37
526
17
23


>Create a Dataframe in which we will store the data

In [6]:
rows = []
for claim in data:
    match = re.search(regex, claim)
    row = [int(match.group(i)) for i in range(1, 6)]
    rows.append(row)
    
columns=['claimID', 'x', 'y', 'width', 'height']
df = pd.DataFrame(rows, columns=columns).set_index('claimID')
df.head()

,x,y,width,height
claimID,,,,
1,37,526,17,23
2,75,649,23,11
3,138,364,29,12
4,370,260,20,17
5,345,512,17,22


### Processing the data

Calculate the maximum row and column values of the cloak. 

In [7]:
df['max_x'] = df.apply(lambda row: row.x + row.width, axis=1)
df['max_y'] = df.apply(lambda row: row.y + row.height, axis=1)
df.head()

,x,y,width,height,max_x,max_y
claimID,,,,,,
1,37,526,17,23,54,549
2,75,649,23,11,98,660
3,138,364,29,12,167,376
4,370,260,20,17,390,277
5,345,512,17,22,362,534


In [8]:
max_x = df.max_x.max()
max_y = df.max_y.max()

santas_cloak = np.zeros((max_y, max_x), dtype=int)
santas_cloak.shape

(999, 999)

### Solving the mystery

In [9]:
def claim_cloak(row):
    santas_cloak[row.y: row.y + row.height, row.x:row.x + row.width, ] += 1
    
df.apply(claim_cloak, axis=1)

MIN_NUMB_CLAIMS = 2 
print("%s square inch is claimed" % np.sum(santas_cloak >= MIN_NUMB_CLAIMS))

119572 square inch is claimed


**That's the right answer! You are one gold star closer to fixing the time stream. [Continue to Part Two](https://adventofcode.com/2018/day/3#part2)**

## Part 2

Amidst the chaos, you notice that exactly one claim doesn't overlap by even a single square inch of fabric with any other claim. If you can somehow draw attention to it, maybe the Elves will be able to make Santa's suit after all!

For example, in the claims above, only claim 3 is intact after all claims are made.

**What is the ID of the only claim that doesn't overlap**?

### Wrong Solution

In [10]:
# this will be the only part of the cloack with ones
result = np.array((santas_cloak == 1), dtype=int)
np.unravel_index(result.argmax(), result.shape)

(0, 759)

> This is wrong it will find the first encounter with 0s, this is not always the first unclaimed recangle, The succeedding example should give more clarity

In [11]:
df.query("y == 0 and x == 759")

,x,y,width,height,max_x,max_y
claimID,,,,,,
563,759,0,12,14,771,14


In [12]:
santas_cloak[0:14,759:759+12]
# this is the first encounter of a 1, but this is not the start index of a whole array

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]])

> **The first value is a 1, but not the whole rectangle**, glad that I didn't submit that

### Correct Solution

In [13]:
df['others_claimed'] = df.apply(lambda row:  np.sum(santas_cloak[row.y: row.y + row.height, row.x:row.x + row.width]) - row.width * row.height, axis=1)

In [14]:
result = df[df['others_claimed'] == 0]
result

,x,y,width,height,max_x,max_y,others_claimed
claimID,,,,,,,
775,826,448,26,27,852,475,0


In [15]:
print("answer: claimid %s has no other claims in its region" % result.index.values[0])

answer: claimid 775 has no other claims in its region


### Verifying solution

In [16]:
santas_cloak[448:448 + 27, 826:826+26]
# this is a whole rectangle with all ones, hence the correct solution

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 